In [1]:
import pandas as pd

In [2]:
# Constantes
PREVIOUS_RESULTS_USED = 5

In [3]:
raw_dfs = {
    'races': pd.read_parquet('.data_parquet/races.parquet'),
    'results': pd.read_parquet('.data_parquet/results.parquet'),
    'driver_standings': pd.read_parquet('.data_parquet/driver_standings.parquet'),
    'constructor_standings': pd.read_parquet('.data_parquet/constructor_standings.parquet'),
    'qualifying': pd.read_parquet('.data_parquet/qualifying.parquet')
}
display(raw_dfs['races'])

,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time,datetime
raceId,,,,,,,,,,,,,,,,,,
1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,2009-03-29 06:00:00
2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,2009-04-05 09:00:00
3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,2009-04-19 07:00:00
4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,2009-04-26 12:00:00
5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,2009-05-10 12:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116,2023,18,69,United States Grand Prix,2023-10-22,19:00:00,https://en.wikipedia.org/wiki/2023_United_Stat...,2023-10-20,17:30:00,2023-10-21,18:00:00,\N,\N,2023-10-20,21:00:00,2023-10-21,22:00:00,2023-10-22 19:00:00
1117,2023,19,32,Mexico City Grand Prix,2023-10-29,20:00:00,https://en.wikipedia.org/wiki/2023_Mexico_City...,2023-10-27,18:30:00,2023-10-27,22:00:00,2023-10-28,17:30:00,2023-10-28,21:00:00,\N,\N,2023-10-29 20:00:00
1118,2023,20,18,São Paulo Grand Prix,2023-11-05,17:00:00,https://en.wikipedia.org/wiki/2023_S%C3%A3o_Pa...,2023-11-03,14:30:00,2023-11-04,14:30:00,\N,\N,2023-11-03,18:00:00,2023-11-04,18:30:00,2023-11-05 17:00:00


In [4]:
races_by_number = raw_dfs['races'][['year', 'round']].sort_values(['year', 'round'])
races_by_number['number'] = range(1, len(races_by_number) + 1)
races_number = races_by_number[['number']]
races_df_with_number = raw_dfs['races'].join(races_by_number['number'], on='raceId')
races_df_with_number = races_df_with_number.set_index('number', append=True)
display(races_df_with_number)

,,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time,datetime
raceId,number,,,,,,,,,,,,,,,,,,
1,804,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,2009-03-29 06:00:00
2,805,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,2009-04-05 09:00:00
3,806,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,2009-04-19 07:00:00
4,807,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,2009-04-26 12:00:00
5,808,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,2009-05-10 12:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116,1097,2023,18,69,United States Grand Prix,2023-10-22,19:00:00,https://en.wikipedia.org/wiki/2023_United_Stat...,2023-10-20,17:30:00,2023-10-21,18:00:00,\N,\N,2023-10-20,21:00:00,2023-10-21,22:00:00,2023-10-22 19:00:00
1117,1098,2023,19,32,Mexico City Grand Prix,2023-10-29,20:00:00,https://en.wikipedia.org/wiki/2023_Mexico_City...,2023-10-27,18:30:00,2023-10-27,22:00:00,2023-10-28,17:30:00,2023-10-28,21:00:00,\N,\N,2023-10-29 20:00:00
1118,1099,2023,20,18,São Paulo Grand Prix,2023-11-05,17:00:00,https://en.wikipedia.org/wiki/2023_S%C3%A3o_Pa...,2023-11-03,14:30:00,2023-11-04,14:30:00,\N,\N,2023-11-03,18:00:00,2023-11-04,18:30:00,2023-11-05 17:00:00


In [5]:
# Exemplo de query a partir de raceId
display(races_df_with_number.xs(1, level='raceId'))
# Exemplo de query a partir de number
display(races_df_with_number.xs(1, level='number'))

,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time,datetime
number,,,,,,,,,,,,,,,,,,
804,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,2009-03-29 06:00:00


,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time,datetime
raceId,,,,,,,,,,,,,,,,,,
833,1950,1,9,British Grand Prix,1950-05-13,\N,http://en.wikipedia.org/wiki/1950_British_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,NaT


In [6]:
import numpy as np

# Parâmetros pensados em um primeiro momento
# - Últimas posições de largada do piloto
# - Últimas posições de chegada do piloto
# - Histórico de classificação do piloto
# - Posição "média" de largada da equipe nas últimas corridas
# - Posição "média" de chegada da equipe nas últimas corridas
# - Histórico de classificação da equipe
result_df = raw_dfs["results"]
driver_standings_df = raw_dfs["driver_standings"]
constructor_standings_df = raw_dfs["constructor_standings"]

def query_info_for_race(raceId: int, driverId: int):
    race_not_found = raceId not in races_df_with_number.index.get_level_values(0)
    if race_not_found:
        raise ValueError("Race is not present in the dataset")
    race_row = races_df_with_number.xs(raceId, level='raceId')
    race_number = race_row.index[0]
    last_n_races = range(race_number - PREVIOUS_RESULTS_USED, race_number)
    last_n_racesId = races_df_with_number.loc[(
        slice(None),
        last_n_races
    ), :].index.get_level_values(0)
    last_grids = []
    last_finishes = []
    last_standings = []
    last_grids_team = []
    last_finishes_team = []
    last_standings_team = []
    for prev_raceId in last_n_racesId:
        prev_race_results = result_df.loc[prev_raceId]
        drivers_in_race_driverId = prev_race_results.index
        did_not_result = driverId not in drivers_in_race_driverId
        if did_not_result:
            last_grids.append(None)
            last_finishes.append(None)
        else:
            this_driver_result = prev_race_results.loc[driverId]
            this_driver_constructorId = this_driver_result['constructorId']
            this_constructor_result = prev_race_results.query(
                f"constructorId == {this_driver_constructorId}"
            )
            last_grids.append(this_driver_result['grid'])
            prev_race_position = this_driver_result['position']
            last_finishes.append(None if np.isnan(prev_race_position) else int(prev_race_position))
            this_constructor_mean_grid = this_constructor_result['grid'].mean()
            last_grids_team.append(this_constructor_mean_grid)
            this_constructor_mean_position = this_constructor_result['position'].dropna().mean()
            last_finishes_team.append(this_constructor_mean_position)
            this_constructor_standing = constructor_standings_df.loc[(
                prev_raceId,
                this_driver_constructorId
            )]["position"]
            last_standings_team.append(this_constructor_standing)
        prev_race_standings = driver_standings_df.loc[prev_raceId]
        did_not_have_standing = driverId not in prev_race_standings.index
        if did_not_have_standing:
            last_standings.append(None)
        else:
            last_standings.append(
                prev_race_standings.loc[driverId]["position"]
            )
    display('hist_grid', last_grids)
    display('hist_finish', last_finishes)
    display('hist_standings', last_standings)
    display('hist_grids_team', last_grids_team)
    display('hist_finish_team', last_finishes_team)
    display('hist_standings_team', last_standings_team)


query_info_for_race(10, 1)

'hist_grid'

[14, 19, 16, 18, 5]

'hist_finish'

[9, 12, 13, 16, 18]

'hist_standings'

...

[16.0, 13.0, 15.0, 15.5, 5.5]

'hist_finish_team'

[9.0, 12.0, 13.5, 16.0, 13.0]

'hist_standings_team'

[4, 5, 5, 6, 6]